<a href="https://colab.research.google.com/github/carlosbkm/ADaSci-Agentic-AI-Architect-labs/blob/main/Introduction_to_CrewAI_and_its_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Agent Customer Support System

In this walkthrough, we’ll demonstrate how to build a multi-agent support workflow using CrewAI, a lightweight framework for designing collaborative AI systems.
You’ll learn how to:


*   Define role-specific agents with goals, backstories, and communication styles
*   Set up tools for agents to access external knowledge and verify facts
*   Orchestrate structured task pipelines with delegation and review flows
*   Use CrewAI memory to enable more informed interactions across tasks


By the end, you’ll have a fully functional AI-powered support team that can answer customer questions, self-check for quality, and deliver a polished final response — all within a transparent, traceable multi-agent system.


In [5]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 huggingface_hub

In [11]:
!pip install crewai crewai_tools langchain_community huggingface_hub

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [13]:
from crewai import Agent, Task, Crew

In [27]:
import requests
from google.colab import userdata

hf_token = userdata.get("HF_API_TOKEN")

if not hf_token:
    raise ValueError("❌ No token found in Runtime > Variables. Please set HF_API_TOKEN first.")

headers = {"Authorization": f"Bearer {hf_token}"}

print("🔎 Checking token permissions...\n")

r = requests.get("https://huggingface.co/api/whoami-v2", headers=headers)
print("Status code:", r.status_code)
print("Response JSON:\n", r.json() if r.ok else r.text)


🔎 Checking token permissions...

Status code: 200
Response JSON:
 {'type': 'user', 'id': '68ee35eff0b9090824368237', 'name': 'carlosbkm', 'fullname': 'Carlos Garcia', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/0eE4BUPA-mxnknkUCwOpm.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Inference API token', 'role': 'fineGrained', 'createdAt': '2025-10-14T12:56:21.620Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '68ee35eff0b9090824368237', 'type': 'user', 'name': 'carlosbkm'}, 'permissions': ['inference.serverless.write', 'inference.endpoints.infer.write']}]}}}}


In [25]:
import requests, os
from google.colab import userdata

hf_token = userdata.get("HF_API_TOKEN")
if not hf_token:
    raise ValueError("❌ Please set HF_API_TOKEN in Runtime > Variables")

def hf_generate(prompt,
                # CHANGE: Use a known, reliably free model like 'gpt2'
                model="gpt2",
                temperature=0.7,
                max_new_tokens=512):
    """Query the free Hugging Face hosted inference API directly."""
    # The URL structure remains the same
    url = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {hf_token}"}
    payload = {
        "inputs": prompt,
        "parameters": {"temperature": temperature,
                       "max_new_tokens": max_new_tokens},
    }
    r = requests.post(url, headers=headers, json=payload)
    r.raise_for_status() # This will now likely succeed
    data = r.json()

    # Note: GPT2 is a Causal LM, so the output may need different handling
    # depending on the full response structure, but this general function
    # should still extract the generated text.
    if isinstance(data, list):
        # For text generation models, the response is often a list of one dict
        return data[0].get("generated_text", "")
    return data.get("generated_text", "")

# 🔍 quick test
print(hf_generate("Explain briefly what Reinforcement Learning is."))

HTTPError: 404 Client Error: Not Found for url: https://api-inference.huggingface.co/models/gpt2

**Role playing, Focus and coperation**

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
     "You work at crewAI (https://crewai.com) and "
     " are now working on providing "
		 "support to {customer}, a super important customer "
     " for your company."
		 "You need to make sure that you provide the best support!"
		 "Make sure to provide full complete answers, "
     " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
	llm=hf_chat
)



*   By not setting allow_delegation=False, allow_delegation takes its default value of being True.

*   This means the agent can delegate its work to another agent which is better suited to do a particular task.





In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
    "are now working with your team "
		"on a request from {customer} ensuring that "
    "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
    "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm=hf_chat
)



*   **Role Playing:** Both agents have been given a role, goal and backstory.
*   **Focus:** Both agents have been prompted to get into the character of the roles they are playing.
*   **Cooperation:** Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

# Tools, Guardrails and Memory

**Tools**

Import crewAI tools

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	      "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		    "Make sure to use everything you know "
        "to provide the best support possible."
		    "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	      "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		    "leaving no questions unanswered, and maintain a helpful and friendly "
		    "tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		    "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		    "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		    "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		    "relevant feedback and improvements.\n"
		    "Don't be too formal, we are a chill and cool company "
	      "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Kshitij Upadhyay",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

**Display the result in Markdown**

In [ ]:
from IPython.display import Markdown
Markdown(result)